In [2]:
import sys
import os

current_path = os.path.abspath('')
parent_path = os.path.dirname(current_path)

if parent_path not in sys.path:
    sys.path.insert(0, parent_path)

import geopandas as gpd
import pandas as pd
import numpy as np
import statsmodels.api as sm

from smoother import ConstantTerm, LinearTerm, DistanceWeighting, SpatialWeightSmoother
from gass import GASS

# Naive Strategy
$y_{sd} = \beta_{cap} Capacity_{sd} + \beta_{job} lgJob_{sd} + \beta_{poi} POI_{sd}$

In [3]:
# read data
cb_study_area = gpd.read_file("../data/cb_study_area.geojson")
cbdf = cb_study_area[cb_study_area.trips_count != 0]

In [4]:
Xcap_cb = cbdf.capacity.values.reshape(-1,1)
Xjob_cb = cbdf.job19.values.reshape(-1,1)
Xpoi_cb = cbdf.pois_count.values.reshape(-1,1)

In [5]:
# Create X
Xcap_cb = cbdf.capacity.values.reshape(-1,1)
lgXjob_cb = np.log(cbdf.job19.values.reshape(-1,1))
lgXjob_cb = np.where(lgXjob_cb < 0, 0, lgXjob_cb) # make sure that np.log(0) is 0
Xpoi_cb = cbdf.pois_count.values.reshape(-1,1)

# Standarize X
Xcap_cb_sd = (Xcap_cb-np.mean(Xcap_cb))/np.std(Xcap_cb)
lgXjob_cb_sd = (lgXjob_cb-np.mean(lgXjob_cb))/np.std(lgXjob_cb)
Xpoi_cb_sd = (Xpoi_cb-np.mean(Xpoi_cb))/np.std(Xpoi_cb)

X_cb_sd = np.hstack((Xcap_cb_sd, lgXjob_cb_sd, Xpoi_cb_sd))

# Standardize y
y_cb = cbdf.trips_count.values.reshape(-1,1)
y_cb_sd = (y_cb-np.mean(y_cb))/np.std(y_cb)

<ipython-input-5-f78e78d3d9c7>:3: RuntimeWarning: divide by zero encountered in log
  lgXjob_cb = np.log(cbdf.job19.values.reshape(-1,1))


In [6]:
ols_cb_sd = sm.OLS(y_cb_sd, X_cb_sd)
print(ols_cb_sd.fit().summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.279
Model:                            OLS   Adj. R-squared (uncentered):              0.274
Method:                 Least Squares   F-statistic:                              65.93
Date:                Wed, 22 May 2024   Prob (F-statistic):                    4.61e-36
Time:                        11:30:42   Log-Likelihood:                         -646.64
No. Observations:                 515   AIC:                                      1299.
Df Residuals:                     512   BIC:                                      1312.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

# GASS

$y_{sd} = \beta_{cap} Capacity_{sd} + \beta_{job} s(lgJob_{sd}, \sigma_{job}) + \beta_{poi} s(POI_{sd}, \sigma_{POI})$

In [7]:
# read data
stations_study_area = gpd.read_file("../data/stations_study_area.geojson") # Stations within the study area (i.e., all bike stations in Brooklyn)
job_cb_study_area = gpd.read_file("../data/job_cb_study_area.geojson") # NO of jobs within the study area (i.e., all census blocks within the study area)
pois_study_area = gpd.read_file("../data/pois_study_area.geojson") # POIs within the study area 

In [8]:
trips_stn = stations_study_area[['StationID', 'end_count', 'capacity']].rename(columns = {'end_count':'Trips_End', 'capacity':'Capacity'})

# take logrithm of job
job_cb_centroid_study_area = job_cb_study_area.set_geometry(job_cb_study_area.centroid)
job_cb_centroid_study_area['job19'] = np.log(job_cb_centroid_study_area['job19'].values)
job_cb_centroid_study_area['job19'] = np.where(job_cb_centroid_study_area['job19'] < 0 , 0.0, job_cb_centroid_study_area['job19'])

<ipython-input-8-7579ab718139>:5: RuntimeWarning: divide by zero encountered in log
  job_cb_centroid_study_area['job19'] = np.log(job_cb_centroid_study_area['job19'].values)


In [9]:
# Necessary data for smoothing job and pois
stn4merge_gdf = stations_study_area.copy()[['StationID', 'capacity', 'geometry']].rename(columns = {'StationID': 'ID', 'capacity' : 'attr' }) 
job4merge_gdf = job_cb_centroid_study_area.copy()[['CBID', 'job19', 'geometry']].rename(columns = {'CBID': 'ID', 'job19' : 'attr' }) 
pois4merge_gdf = pois_study_area.copy()

job_stn = pd.concat([stn4merge_gdf, job4merge_gdf], ignore_index=True).set_geometry('geometry').set_crs(cb_study_area.crs)
job_df = job_cb_centroid_study_area[['CBID', 'job19']]

pois_stn = pd.concat([stn4merge_gdf, pois4merge_gdf], ignore_index=True).set_geometry('geometry').set_crs(cb_study_area.crs)
pois_df = pois_study_area[['ID', 'attr']]

In [10]:
# Construct smoothers for Xcap, Xjob and Xpois
lin_stn_sd = LinearTerm(trips_stn, 2, standard = True) # capacity 
dw_job_stn_sd = DistanceWeighting(trips_stn, job_stn, job_df, [0,0,0,1], standard = True) # smoothing NO. of jobs of each census block
dw_pois_stn_sd = DistanceWeighting(trips_stn, pois_stn, pois_df, [0,0,0,1], standard = True) #smoothing POIs of restaurants, bars, cafes, and shopping malls within the study area

In [11]:
dw_job_stn_sd.set_searching_range(-5, 0)
dw_pois_stn_sd.set_searching_range(-5, 0)

In [12]:
# Standarize y
y_stn = trips_stn.Trips_End.values.reshape(-1,1)
y_stn_sd = (y_stn - np.mean(y_stn))/(np.std(y_stn))

In [13]:
# Fit GASS model
gass_stn_sd = GASS(y_stn_sd, lin_stn_sd, dw_job_stn_sd, dw_pois_stn_sd, constant = False)
gass_stn_sd.fit_Gaussian()

In [14]:
gass_stn_sd.coefficients # Xcap, lgXjob, Xpoi (from top to bottom)

array([[0.23534496],
       [0.130504  ],
       [0.57876038]])

In [15]:
gass_stn_sd.sigmas # Sigmas of s(lgXjob) and s(Xpoi)

[-4.98, -1.05]

In [16]:
# Compute inference statistics of GASS model
gass_stn_sd.inference_Gaussian()

In [17]:
# 95% Confidence Intervals of Coefficients # Xcap, lgXjob, Xpoi (from left to right)
gass_stn_sd.CI_betas

[(0.1738, 0.2969), (0.0706, 0.1904), (0.5173, 0.6402)]

In [18]:
# P-values of Coefficients # Xcap, lgXjob, Xpoi (from left to right)
gass_stn_sd.pvals

array([2.36699549e-13, 2.23164673e-05, 0.00000000e+00])

In [19]:
# R-squared Value of GASS Model
gass_stn_sd.R_squared

0.47878345900071173

In [20]:
from datetime import datetime
start1 = datetime.now()
gass_stn_sd.calculate_AWCI_sigmas(interval = 0.1)
end1 = datetime.now()

In [21]:
# end1-start1 is to track running time
# Akaike Weight Confience Intervals of Sigmas of s(lgXjob) and s(Xpoi)
print(end1-start1, gass_stn_sd.AWCI_sigmas)

0:00:10.099982 [(-5.0, -2.18), (-1.1, -0.95)]


In [22]:
start2 = datetime.now()
gass_stn_sd.calculate_RBCI_sigmas(max_iter = 100)
end2 = datetime.now()

In [23]:
# Ressidual Bootstrap Confience Intervals of Sigmas of s(Xjob) and s(Xpoi) 
print(end2-start2, gass_stn_sd.RBCI_sigmas)

6:55:46.331531 [(-4.98, -2.6333), (-1.12, -0.93)]
